Resource: [```online-retail.csv```](https://drive.google.com/file/d/1O1oJtpEu-u6s6xTu7seQfnNcQjYCt0GF/view?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time


In [2]:
df = pd.read_csv('online_retail.csv', parse_dates = ['InvoiceDate'])
df.head()
#uploading the data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
#How many columns with NA?
df.isna().sum()[df.isna().sum() > 0].count()

2

In [6]:
#Where is NA?
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [7]:
#What historical period do we have data for?
print(df['InvoiceDate'].min().year, df['InvoiceDate'].max().year, sep=', ')

2010, 2011


In [8]:
#What are the min and the max UnitPrice?
print(df.UnitPrice.min(), df.UnitPrice.max(), sep=', ')

-11062.06, 38970.0


In [9]:
#What is the description for transactions with a negative value?
sorted(df[df.UnitPrice < 0]['Description'].values)

['Adjust bad debt', 'Adjust bad debt']

In [10]:
#How many transactions with zero price are in the table? And with passes on the spot prices?
print(df[df.UnitPrice == 0].UnitPrice.count(), df.UnitPrice.isnull().sum(), sep=', ')

2515, 0


#### Решение

In [11]:
#create the dataframe for future analysis
new_df = df[(df.Quantity > 0)&(df.UnitPrice > 0)&(df.CustomerID)].dropna()
new_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
#calculate the full cost for the product
new_df['Price'] = new_df.UnitPrice * new_df.Quantity
#print the min and the max full price
print(round(new_df.Price.min()), round(new_df.Price.max()), sep=', ')


0, 168470


In [13]:
#calculate the cost of the purchases
sort_df = new_df.groupby('InvoiceNo').agg({'Price':'sum'}).reset_index().sort_values('Price', ascending=False).head(3)
#print three most expensive purchases
print(round(sort_df.Price.iloc[0]), round(sort_df.Price.iloc[1]), round(sort_df.Price.iloc[2]), sep=', ')

168470, 77184, 38970


In [14]:
#Which product made the most revenue?
new_df[new_df.InvoiceNo == sort_df.InvoiceNo.iloc[0]].Description.values[0]

'PAPER CRAFT , LITTLE BIRDIE'

In [15]:
# Create a new table purchases, in which each row will correspond to a separate purchase, 
# with columns InvoiceNo, InvoiceDate, Price, CustomerID, Country.
# Since the InvoiceDate can be different for different transactions within the same purchase, 
# when grouping, take the earliest date. We will also sort it by date in ascending order.
purchases = new_df.groupby('InvoiceNo').agg({'InvoiceDate':'first', 'Price':'sum', 
                                             'CustomerID':'first', 'Country':'first'}).reset_index().sort_values('InvoiceDate')
purchases.head()

,InvoiceNo,InvoiceDate,Price,CustomerID,Country
0,536365,2010-12-01 08:26:00,139.12,17850.0,United Kingdom
1,536366,2010-12-01 08:28:00,22.20,17850.0,United Kingdom
2,536367,2010-12-01 08:34:00,278.73,13047.0,United Kingdom
3,536368,2010-12-01 08:34:00,70.05,13047.0,United Kingdom
4,536369,2010-12-01 08:35:00,17.85,13047.0,United Kingdom


In [16]:
#investigate whether our customers' purchases are increasing or decreasing 
# after their first purchase on the site.
print(int(round(purchases.groupby(['CustomerID']).agg({'Price':'first'}).mean())), round(purchases.Price.mean()), sep=', ')

425, 481


In [17]:
# What day of the week did you have the most purchases?
new_df['Weekday'] = [i.dayofweek for i in new_df.InvoiceDate]
weekdays={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}    
new_df['Weekday'] = new_df.Weekday.map(weekdays)
new_df.groupby('Weekday').agg({'InvoiceNo':'count'}).reset_index().sort_values('InvoiceNo', ascending=False).head(1)['Weekday'].values[0]

'Thursday'

In [18]:
#In which year and month was the revenue highest?
new_df['Invoice_Year'] = new_df.InvoiceDate.dt.year
new_df['Invoice_Month'] = new_df.InvoiceDate.dt.month
month_year = new_df.groupby(['Invoice_Year', 'Invoice_Month']).agg({'Price':'sum'}).sort_values('Price', ascending=False).reset_index()
print(month_year.Invoice_Year[0], month_year.Invoice_Month[0], sep=', ')


2011, 11


In [19]:
# Which country had the largest percentage growth in monthly sales 
# when comparing March 2011 to September 2011? How many percent was this growth?
df_19 = new_df[new_df.Invoice_Year == 2011][(new_df.Invoice_Month == 3) | (new_df.Invoice_Month == 9)][new_df.Price > 0]
df_191=df_19.groupby(['Country', 'Invoice_Month']).agg({'Price':'sum'}).reset_index()
country=list(df_191['Country'].values)
for i in country:
    if country.count(i) != 2:
        country.remove(i)
country
df_192=round(df_191[df_191.Country.isin(country)].groupby('Country').agg({'Price':'pct_change'})*100)
df_192['Country']=country
pct_growing=df_192.dropna(subset=['Price']).sort_values('Price', ascending=False).head(1)
print(pct_growing.Country.values[0], int(pct_growing.Price.values), sep=', ')

Norway, 586


<ipython-input-19-4d24bb089ad6>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_19 = new_df[new_df.Invoice_Year == 2011][(new_df.Invoice_Month == 3) | (new_df.Invoice_Month == 9)][new_df.Price > 0]
<ipython-input-19-4d24bb089ad6>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_19 = new_df[new_df.Invoice_Year == 2011][(new_df.Invoice_Month == 3) | (new_df.Invoice_Month == 9)][new_df.Price > 0]


#### Решение

In [20]:
# How many customers have made purchases from at least two different countries?
count_countries=new_df.groupby('CustomerID').agg({'Country':'nunique'})
len(count_countries[count_countries >=2].dropna())

8

In [21]:
# what pair of different products is most often found in various purchases in Italy, 
# and in how many purchases this occurs
Italy=pd.DataFrame(new_df[new_df.Country =='Italy'])
pairs=Italy.merge(Italy, on='InvoiceNo', how='left').drop_duplicates().groupby(['Description_x', 'Description_y']).agg({'InvoiceNo':'count'}).sort_values('InvoiceNo', ascending=False).reset_index()
pair=pairs[pairs.Description_x != pairs.Description_y][::2]
pair[pair.InvoiceNo == pair.InvoiceNo.max()]

,Description_x,Description_y,InvoiceNo
12,PLASTERS IN TIN CIRCUS PARADE,PLASTERS IN TIN WOODLAND ANIMALS,5
20,TOY TIDY SPACEBOY,TOY TIDY PINK POLKADOT,5


##### 